In [1]:
!pip3 install nltk 

In [2]:
#mount drive first
%mkdir /content/chatbot_smart_garden
%cp /content/drive/My\ Drive/Colab\ Notebooks/chatbot_smart_garden/* /content/chatbot_smart_garden
%cd /content/chatbot_smart_garden

/content/chatbot_smart_garden


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

Using TensorFlow backend.


In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents) #list of classes and documents to add all the intents associated with patterns
print(classes)
print(words)

[(['Annaffia', 'pomodori'], 'annaffia_pomodori'), (['pomodori'], 'annaffia_pomodori'), (['I', 'pomodori', 'sono', 'secchi'], 'annaffia_pomodori'), (['pomodori', 'annaffia'], 'annaffia_pomodori'), (['Annaffia', 'rose'], 'annaffia_rose'), (['rose'], 'annaffia_rose'), (['Le', 'rose', 'sono', 'secche'], 'annaffia_rose'), (['rose', 'annaffia'], 'annaffia_rose'), (['Annaffia', 'geranei'], 'annaffia_geranei'), (['geranei'], 'annaffia_geranei'), (['I', 'geranei', 'sono', 'secchi'], 'annaffia_geranei'), (['geranei', 'annaffia'], 'annaffia_geranei'), (['Annaffia', 'magnolie'], 'annaffia_magnolie'), (['magnolie'], 'annaffia_magnolie'), (['Le', 'magnolie', 'sono', 'secche'], 'annaffia_magnolie'), (['magnolie', 'annaffia'], 'annaffia_magnolie'), (['Annaffia', 'viole'], 'annaffia_viole'), (['viole'], 'annaffia_viole'), (['Le', 'viole', 'sono', 'secche'], 'annaffia_viole'), (['viole', 'annaffia'], 'annaffia_viole')]
['annaffia_pomodori', 'annaffia_rose', 'annaffia_geranei', 'annaffia_magnolie', 'anna

In [6]:
#We can convert words into the lemma form so that we can reduce all the canonical words. 
#For example, the words play, playing, plays, played, etc. will all be replaced with play.

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
print(words)

['annaffia', 'geranei', 'i', 'le', 'magnolie', 'pomodori', 'rose', 'secche', 'secchi', 'sono', 'viole']


In [7]:
# sort classes
classes = sorted(list(set(classes)))
print(classes)

['annaffia_geranei', 'annaffia_magnolie', 'annaffia_pomodori', 'annaffia_rose', 'annaffia_viole']


In [8]:
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

20 documents
5 classes ['annaffia_geranei', 'annaffia_magnolie', 'annaffia_pomodori', 'annaffia_rose', 'annaffia_viole']
11 unique lemmatized words ['annaffia', 'geranei', 'i', 'le', 'magnolie', 'pomodori', 'rose', 'secche', 'secchi', 'sono', 'viole']


In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
    # class matrix: output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x)
print(train_y)

[[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0,

In [0]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [0]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
20/20 [==============================] - 2s 87ms/step - loss: 1.6745 - accuracy: 0.2000
Epoch 2/200
20/20 [==============================] - 0s 671us/step - loss: 1.5631 - accuracy: 0.2000
Epoch 3/200
20/20 [==============================] - 0s 681us/step - loss: 1.4993 - accuracy: 0.2000
Epoch 4/200
20/20 [==============================] - 0s 736us/step - loss: 1.5054 - accuracy: 0.3000
Epoch 5/200
20/20 [==============================] - 0s 623us/step - loss: 1.4036 - accuracy: 0.4500
Epoch 6/200
20/20 [==============================] - 0s 622us/step - loss: 1.3814 - accuracy: 0.5500
Epoch 7/200
20/20 [==============================] - 0s 720us/step - loss: 1.3016 - accuracy: 0.7000
Epoch 8/200
20/20 [==============================] - 0s 682us/step - loss: 1.2608 - accuracy: 0.5500
Epoch 9/200
20/20 [==============================] - 0s 662us/step - loss: 1.2297 - accuracy: 0.7000
Epoch 10/200
20/20 [==============================] - 0s 685us/step - loss: 1.0787 - accurac

In [13]:
%ls

chatbot_model.h5  classes.pkl  intents.json  words.pkl


In [14]:
!pip3 install telepot

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 1.2MB 8.7MB/s 
     |████████████████████████████████| 153kB 33.3MB/s 
     |████████████████████████████████| 256kB 42.1MB/s 
  Created wheel for telepot: filename=telepot-12.7-cp36-none-any.whl size=57963 sha256=0e3b111ec95a328772764fd1220e4dcd4f84c44c7f5a8118afe69af4168ad6f5
  Stored in directory: /root/.cache/pip/wheels/65/17/50/983034e27e6b5d8dfc497a51b5b13d970d8018a83f47e6513a
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=5f303af48f4ee119791f1d66511a6069214487c59ed1e19238d31ee21e7b6365
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built telepot idna-ssl


In [0]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

#from keras.models import load_model
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def predict_response(msg):
  ints = predict_class(msg)
  res = getResponse(ints, intents)
  return res

def on_chat_message(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)
    if content_type == 'text':
        #name = msg["first_name"]
        txt = msg['text']
        res=predict_response(txt)
        bot.sendMessage(chat_id, res)

In [16]:
import telepot

TOKEN = '<TOKEN>'

bot = telepot.Bot(TOKEN)
bot.message_loop(on_chat_message)

print('Listening ...')

import time
while 1:
    time.sleep(10)

Listening ...


KeyboardInterrupt: ignored